# Langchain

3 utilisations principales:

- Components (Wrappers, prompt templates, RAG)
- Chains (Assemble components to solve specific task)
- Agents (Allow LLM to interact with is environement)

In [ ]:
# Install the requirements
# uv pip install python-dotenv langchain

In [25]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Invoker le model

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-small-latest",
    temperature=0,
    max_retries=2,
    )

## Generation simple et sans contexte

In [32]:
result = llm.predict('Hi im Ethan and u ? ')
print(result)

Hello Ethan! I'm an AI assistant. I don't have a name or personal experiences, but I'm here to help answer your questions, provide information, or just chat if you're up for it. How can I assist you today?


## Avec Contexte

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant named Unity",
    ),
    ("human", "Whats your name ?"),
]

AIMessage(content="Hello! I'm Unity. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 18, 'total_tokens': 33, 'completion_tokens': 15}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-9b0e0a1d-b8b0-474e-92e1-6922aa5b9797-0', usage_metadata={'input_tokens': 18, 'output_tokens': 15, 'total_tokens': 33})

In [ ]:
ai_msg = llm.invoke(messages)
print(ai_msg.content)

Hello! I'm Unity. How can I assist you today?


## Chat Model
https://python.langchain.com/v0.1/docs/modules/model_io/chat/quick_start/

In [39]:
from langchain_mistralai import ChatMistralAI
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

PROMPT_COUNTRY_INFO = "Provide information about {country}."


def main():
    chat = ChatMistralAI(model="mistral-small-latest")

    country = input('Name a country: ')
    
    message = HumanMessagePromptTemplate.from_template(template=PROMPT_COUNTRY_INFO)
    chat_prompt = ChatPromptTemplate.from_messages(messages=[message])
    chat_prompt_with_values = chat_prompt.format_prompt(country=country)

    response = chat(chat_prompt_with_values.to_messages())
    print(response.content)




In [40]:
main()

France, officially known as the French Republic, is a country located in Western Europe. Here are some key points of information about France:

### Geography:
- **Location:** Western Europe, bordered by Belgium, Luxembourg, Germany, Switzerland, Italy, Monaco, Spain, Andorra, and the Mediterranean Sea.
- **Capital:** Paris
- **Major Cities:** Marseille, Lyon, Toulouse, Nice, Nantes, Montpellier, Strasbourg, Lille, Bordeaux.
- **Landmarks:** Eiffel Tower, Louvre Museum, Palace of Versailles, Mont Blanc, French Riviera, Champs-Élysées, Notre-Dame Cathedral.

### Government and Politics:
- **Form of Government:** Semi-presidential republic
- **Head of State:** President (currently Emmanuel Macron as of my last update in October 2023)
- **Legislature:** Bicameral Parliament consisting of the National Assembly and the Senate
- **Political Parties:** Includes a range of parties from left to right, such as La République En Marche!, The Republicans, National Rally, and Socialist Party.

### Ec

### Convert the output to json format

In [56]:
from langchain_mistralai import ChatMistralAI
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Country(BaseModel):
    capital: str = Field(description='Capital of the country')
    name: str = Field(description='Name of the country')

PROMPT_COUNTRY_INFO = """
Provide information about {country}.
{format_instructions}
"""

def main():
    chat = ChatMistralAI(model="mistral-small-latest")
    parser = PydanticOutputParser(pydantic_object=Country)

    country = input('Name a country: ')

    message = HumanMessagePromptTemplate.from_template(template=PROMPT_COUNTRY_INFO)
    chat_prompt = ChatPromptTemplate.from_messages(messages=[message])
    chat_prompt_with_values = chat_prompt.format_prompt(country=str(country), format_instructions=parser.get_format_instructions())

    response = chat(chat_prompt_with_values.to_messages())
    data = parser.parse(response.content)
    # print(response.content)
    print(data.capital)



In [57]:
main()

Paris


# Exemple de chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)